In [707]:
import torch
import torch.nn as nn

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import os
import sys
import time
import warnings
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
import matplotlib as mpl

import tensorflow as tf
from tensorflow import keras


torch.cuda.is_available()

True

In [708]:
x = torch.rand(5, 3)
print(x)

tensor([[0.9493, 0.7650, 0.1776],
        [0.5825, 0.2832, 0.2370],
        [0.5088, 0.6017, 0.6505],
        [0.3899, 0.5499, 0.0422],
        [0.6392, 0.9158, 0.9977]])


In [709]:

import os.path


train_all = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

NUMERIC_FEATURE_NAMES = [
    "Age",
    "RoomService",
    "FoodCourt",
    "ShoppingMall",
    "Spa",
    "VRDeck",
]

CATEGORICAL_FEATURE_NAMES = ["HomePlanet","Destination"]


train_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [710]:
test_data_file = "test_data.csv"
test.to_csv(test_data_file, index=False, header=True)

train_all["CryoSleep"].replace([True,False], ["1", "0"], inplace=True)
train_all["VIP"].replace([True,False], ["1", "0"], inplace=True)
test["CryoSleep"].replace([True,False], ["1", "0"], inplace=True)
test["VIP"].replace([True,False], ["1", "0"], inplace=True)

train_all["Relation"] = train_all["PassengerId"].str.split("_").str[1].astype(float)
test["Relation"] = test["PassengerId"].str.split("_").str[1].astype(float)

# train_all["Deck"] = train_all["Cabin"].str[0]
# test["Deck"] = test["Cabin"].str[0]

# train_all["Side"] = train_all["Cabin"].str[2]
# test["Side"] = test["Cabin"].str[2]


train_all.drop(['PassengerId', 'Cabin','Name'],axis =1 , inplace=True)
test.drop(['PassengerId', 'Cabin','Name'],axis = 1 ,inplace=True)
train_all.Transported.replace([True,False], ["1", "0"], inplace=True)

In [711]:

train_all["Age"].fillna(train_all["Age"].mean(skipna=True), inplace=True)
test["Age"].fillna(test["Age"].mean(skipna=True), inplace=True)

train_all["RoomService"].fillna(train_all["RoomService"].mean(skipna=True), inplace=True)
test["RoomService"].fillna(test["RoomService"].mean(skipna=True), inplace=True)

train_all["FoodCourt"].fillna(train_all["FoodCourt"].mean(skipna=True), inplace=True)
test["FoodCourt"].fillna(test["FoodCourt"].mean(skipna=True), inplace=True)

train_all["ShoppingMall"].fillna(train_all["ShoppingMall"].mean(skipna=True), inplace=True)
test["ShoppingMall"].fillna(test["ShoppingMall"].mean(skipna=True), inplace=True)

train_all["Spa"].fillna(train_all["Spa"].mean(skipna=True), inplace=True)
test["Spa"].fillna(test["Spa"].mean(skipna=True), inplace=True)

train_all["VRDeck"].fillna(train_all["VRDeck"].mean(skipna=True), inplace=True)
test["VRDeck"].fillna(test["VRDeck"].mean(skipna=True), inplace=True)

#  ["HomePlanet","CryoSleep","Destination","VIP"]

train_all["HomePlanet"].fillna('Kepler', inplace=True)
test["HomePlanet"].fillna('Kepler', inplace=True)

train_all["CryoSleep"].fillna('0', inplace=True)
test["CryoSleep"].fillna('0', inplace=True)

train_all["VIP"].fillna('0', inplace=True)
test["VIP"].fillna('0', inplace=True)

train_all["Destination"].fillna('TRAPPIST-1e', inplace=True)
test["Destination"].fillna('TRAPPIST-1e', inplace=True)

train_all.dropna(inplace = True)
# test.dropna(inplace = True)

In [712]:
for col in CATEGORICAL_FEATURE_NAMES:
    encoder = OneHotEncoder()
    encoded  = encoder.fit_transform(train_all[[col]])
    train_all[col] = encoded.toarray()
  
for col in CATEGORICAL_FEATURE_NAMES:
    encoder = OneHotEncoder()
    encoded  = encoder.fit_transform(test[[col]])
    test[col] = encoded.toarray()

train_all
# cat_encoder = OneHotEncoder()
# housing_cat_encoded  = cat_encoder.fit_transform(train_X[["HomePlanet"]])
# housing_cat_encoded.toarray()
# train_X["HomePlanet"] = housing_cat_encoded.toarray()


,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Relation
0,0.0,0,0.0,39.0,0,0.0,0.0,0.0,0.0,0.0,0,1.0
1,1.0,0,0.0,24.0,0,109.0,9.0,25.0,549.0,44.0,1,1.0
2,0.0,0,0.0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0,1.0
3,0.0,0,0.0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0,2.0
4,1.0,0,0.0,16.0,0,303.0,70.0,151.0,565.0,2.0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.0,0,1.0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,0,1.0
8689,1.0,1,0.0,18.0,0,0.0,0.0,0.0,0.0,0.0,0,1.0
8690,1.0,0,0.0,26.0,0,0.0,0.0,1872.0,1.0,0.0,1,1.0
8691,0.0,0,1.0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,0,1.0


In [713]:
# CSV_HEADER = [
#     "HomePlanet",
#     "Destination",
#     "Age",
#     "RoomService",
#     "FoodCourt",
#     "ShoppingMall",
#     "Spa",
#     "VRDeck",
#     "Transported",
#     ]

train_data_file = "train_data.csv"


In [714]:
# def get_dataset_from_csv(csv_file_path, shuffle=False, batch_size=128):
#     dataset = tf.data.experimental.make_csv_dataset(
#         csv_file_path,
#         batch_size=batch_size,
#         column_names=CSV_HEADER,
#         label_name="Transported",
#         num_epochs=1,
#         shuffle=shuffle
#     )
#     return dataset.cache()

In [715]:
# train_all = pd.read_csv('./train_data.csv')
# test = pd.read_csv('./test_data.csv')

train, valid = train_test_split(train_all, test_size=0.05, random_state=42, shuffle=True)

train_X, train_y = train.drop('Transported', axis=1), train['Transported']
valid_X, valid_y = valid.drop('Transported', axis=1), valid['Transported']

train_X.to_csv("./train_x.csv", index=False, header=False)
train_y.to_csv("./train_y.csv", index=False, header=False)
train_X.to_csv("./valid_x.csv", index=False, header=False)
train_y.to_csv("./valid_y.csv", index=False, header=False)

test.to_csv("./test_v.csv", index=False, header=False)


train_X, train_y = pd.read_csv('./train_x.csv'), pd.read_csv('./train_y.csv')
valid_X, valid_y = pd.read_csv('./valid_x.csv'), pd.read_csv('./valid_y.csv')

test_v = pd.read_csv('./test_v.csv')



train_X.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8257 entries, 0 to 8256
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0.0     8257 non-null   float64
 1   0       8257 non-null   int64  
 2   0.0.1   8257 non-null   float64
 3   31.0    8257 non-null   float64
 4   0.1     8257 non-null   int64  
 5   0.0.2   8257 non-null   float64
 6   1489.0  8257 non-null   float64
 7   68.0    8257 non-null   float64
 8   6.0     8257 non-null   float64
 9   0.0.3   8257 non-null   float64
 10  2.0     8257 non-null   float64
dtypes: float64(9), int64(2)
memory usage: 709.7 KB


In [716]:
test_v.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4276 entries, 0 to 4275
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1.0     4276 non-null   float64
 1   1       4276 non-null   int64  
 2   0.0     4276 non-null   float64
 3   27.0    4276 non-null   float64
 4   0       4276 non-null   int64  
 5   0.0.1   4276 non-null   float64
 6   0.0.2   4276 non-null   float64
 7   0.0.3   4276 non-null   float64
 8   0.0.4   4276 non-null   float64
 9   0.0.5   4276 non-null   float64
 10  1.0.1   4276 non-null   float64
dtypes: float64(9), int64(2)
memory usage: 367.6 KB


In [717]:

# train_X["relation"] = train_X["PassengerId"].str.split("_").str[1].astype(int)
# train_X["relation"]

In [718]:
# train_X["Deck"] = train_X["Cabin"].str[0]

# train_X["Side"] = train_X["Cabin"].str[2]

# train_X.drop(["Cabin", "Name"], axis=1, inplace=True)

# cat_encoder = OneHotEncoder()
# housing_cat_encoded  = cat_encoder.fit_transform(train_X[["Deck"]])
# housing_cat_encoded.toarray()
# train_X["Deck"] = housing_cat_encoded.toarray()

# cat_encoder = OneHotEncoder()
# housing_cat_encoded  = cat_encoder.fit_transform(train_X[["Side"]])
# housing_cat_encoded.toarray()
# train_X["Side"] = housing_cat_encoded.toarray()

# train_X["Deck"]
# train_X["Side"]

In [719]:
test = np.asarray(test).astype('float32')
test

array([[1.000e+00, 1.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [1.000e+00, 0.000e+00, 0.000e+00, ..., 2.823e+03, 0.000e+00,
        1.000e+00],
       [0.000e+00, 1.000e+00, 1.000e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       ...,
       [0.000e+00, 1.000e+00, 1.000e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 5.230e+02,
        1.000e+00],
       [1.000e+00, 1.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00]], dtype=float32)

In [724]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)
model = keras.Sequential([
    keras.layers.BatchNormalization(),
    keras.layers.Dense(8, activation='elu', kernel_initializer='he_normal'),
    keras.layers.Dense(16, activation='elu'),
    keras.layers.Dense(32, activation='elu'),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(64, activation='elu'),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(128, activation='elu'),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['binary_accuracy'])

In [725]:
model.fit(train_X, train_y, epochs=20, validation_data=(valid_X, valid_y))

Epoch 1/20
259/259 [==============================] - 3s 7ms/step - loss: 0.6029 - binary_accuracy: 0.6944 - val_loss: 0.5543 - val_binary_accuracy: 0.7374
Epoch 2/20
259/259 [==============================] - 2s 6ms/step - loss: 0.5489 - binary_accuracy: 0.7334 - val_loss: 0.5296 - val_binary_accuracy: 0.7419
Epoch 3/20
259/259 [==============================] - 2s 6ms/step - loss: 0.5269 - binary_accuracy: 0.7463 - val_loss: 0.5138 - val_binary_accuracy: 0.7506
Epoch 4/20
259/259 [==============================] - 2s 6ms/step - loss: 0.5207 - binary_accuracy: 0.7494 - val_loss: 0.5003 - val_binary_accuracy: 0.7634
Epoch 5/20
259/259 [==============================] - 2s 6ms/step - loss: 0.5079 - binary_accuracy: 0.7558 - val_loss: 0.4912 - val_binary_accuracy: 0.7657
Epoch 6/20
259/259 [==============================] - 2s 6ms/step - loss: 0.5019 - binary_accuracy: 0.7614 - val_loss: 0.4840 - val_binary_accuracy: 0.7658
Epoch 7/20
259/259 [==============================] - 2s 6ms/ste

In [726]:
y_pred = model.predict(test)
y_pred

array([[0.63802904],
       [0.08514839],
       [0.97206736],
       ...,
       [0.973238  ],
       [0.59824705],
       [0.63529736]], dtype=float32)

In [727]:
prediction = pd.DataFrame(y_pred, columns=['Transported'])

prediction["PassengerId"] = pd.read_csv('./test_data.csv')["PassengerId"]

prediction["Transported"].loc[prediction['Transported'] > 0.5] = True
prediction["Transported"].loc[prediction['Transported'] <= 0.5] = False

prediction.to_csv("./submit.csv", index=False, header=True)


# .to_csv('submit.csv')
# prediction

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
